In [1]:
%cd ../..
%pwd
# Should be  <BASE>/code

/home/oli/Research/Joe/agent-goals/code


'/home/oli/Research/Joe/agent-goals/code'

# Random Experiments

The idea here is to generate a lot of PDGs, perform inference on them in different ways / with different parameters, and report (1) timing in formation and (2) training curves. 

Inference algorithms on PDGs to pit against each other:
 * `ExpCone` on full joint distribution;
 * `ExpCone` on clique trees (automatically discovered);
 * `Pytorch` (different learning rates, optimizers: `LBFGS`; `ADAM`)
 * LIR (still needs implementation + schedule)

Of course, we also need to compare against known algorithms; we can generate / use as benchmarks. 
For examples of BNs, I can use

* http://constantinou.info/downloads/bayesys/bayesys_repository.pdf
* the pgmpy example library
* randomly generated BNs

## 1. Make some random factor graphs

In [3]:
from pdg.pdg import PDG
from pdg.rv import Variable as Var, Unit
from pdg.dist import RawJointDist as RJD, CPT

from random import randint, sample

import numpy as np
import networkx as nx
from pgmpy.models import FactorGraph
from pgmpy.factors.discrete import DiscreteFactor

from functools import reduce
from operator import and_


In [6]:

NVARS = 10
NFGs = 15
MIN_FACTORS = 2
MAX_FACTORS = 20

varis = [ Var.alph("X%d"%i, randint(2,4)) for i in range(NVARS)]


fgs = []
pdgs = []

while len(fgs) < NFGs:
	nfactors = randint(MIN_FACTORS, MAX_FACTORS)
	fg = FactorGraph()
	pdg = PDG()

	for j in range(nfactors):
		selection = sample(varis, randint(1,4))
		f = np.random.rand(*[len(v) for v in selection])
		df = DiscreteFactor(
			[v.name for v in selection],
			[len(v) for v in selection],
			f
		)
		fg.add_nodes_from([v.name for v in selection]) # how silly
		# print(fg.nodes)
		fg.add_factors(df)
		# fg.add_nodes_from([fg]) # even sillier
		fg.add_edges_from([(v.name, df) for v in selection])

		pdg += CPT.from_matrix(Unit, reduce(and_, selection), f.reshape(1,-1)/f.sum() )
	
	if(nx.is_connected(fg)):	
		fgs.append(fg)
		pdgs.append(pdg)

In [61]:
from pgmpy.inference import BeliefPropagation
from itertools import chain


fg = fgs[0]
fg.check_model()


True

In [78]:
# print(fg.get_factor_nodes())

# N = set(fgs[0].nodes)
# print(N)
# X = set(v for f in fgs[0].factors for v in f.variables)
# print(X)
print(fg.check_model())
# jt = fg.to_markov_model().to_junction_tree()

print(fg.nodes)
print(fg.edges) 

mm = fg.to_markov_model()
bp = BeliefPropagation(mm)
# bp = BeliefPropagation(jt)
# bp.calibrate()

True
['X13', 'X4', 'X11', 'X18', <DiscreteFactor representing phi(X13:3, X4:4, X11:4, X18:4) at 0x277734b4700>, 'X19', <DiscreteFactor representing phi(X19:4) at 0x277734e1e80>, 'X1', <DiscreteFactor representing phi(X11:4, X1:3, X18:4) at 0x277734e18b0>, 'X5', 'X6', <DiscreteFactor representing phi(X5:3, X6:2) at 0x2777334e820>, 'X2', 'X14', 'X7', <DiscreteFactor representing phi(X2:3, X14:4, X7:4, X6:2) at 0x27773536f70>, 'X16', <DiscreteFactor representing phi(X6:2, X7:4, X16:2) at 0x27769b7a220>, <DiscreteFactor representing phi(X7:4) at 0x277735368b0>]
[('X13', <DiscreteFactor representing phi(X13:3, X4:4, X11:4, X18:4) at 0x277734b4700>), ('X4', <DiscreteFactor representing phi(X13:3, X4:4, X11:4, X18:4) at 0x277734b4700>), ('X11', <DiscreteFactor representing phi(X13:3, X4:4, X11:4, X18:4) at 0x277734b4700>), ('X11', <DiscreteFactor representing phi(X11:4, X1:3, X18:4) at 0x277734e18b0>), ('X18', <DiscreteFactor representing phi(X13:3, X4:4, X11:4, X18:4) at 0x277734b4700>), ('X

ValueError: No sepset found between these two edges.

In [81]:

from matplotlib import pyplot as plt
import networkx as nx

TypeError: 'int' object is not iterable

In [94]:
gr = nx.Graph(fg.edges())
[*nx.connected_components(gr)]

[{<DiscreteFactor representing phi(X13:3, X4:4, X11:4, X18:4) at 0x277734b4700>,
  <DiscreteFactor representing phi(X11:4, X1:3, X18:4) at 0x277734e18b0>,
  'X1',
  'X11',
  'X13',
  'X18',
  'X4'},
 {<DiscreteFactor representing phi(X19:4) at 0x277734e1e80>, 'X19'},
 {<DiscreteFactor representing phi(X6:2, X7:4, X16:2) at 0x27769b7a220>,
  <DiscreteFactor representing phi(X5:3, X6:2) at 0x2777334e820>,
  <DiscreteFactor representing phi(X2:3, X14:4, X7:4, X6:2) at 0x27773536f70>,
  <DiscreteFactor representing phi(X7:4) at 0x277735368b0>,
  'X14',
  'X16',
  'X2',
  'X5',
  'X6',
  'X7'}]

In [1]:
gr

NameError: name 'gr' is not defined